In [1]:
from pathlib import Path

import automol

ade_path = Path("autode")
opt_path = Path("opt")
geo_name = Path("geom.xyz")
zma_name = Path("geom.zmat")
ene_name = Path("sp.ene")
zrxn_name = Path("zmat.r.yaml")

In [2]:
# Read geometry and prepare TS graph
geo = automol.geom.from_xyz_string((ade_path / geo_name).read_text())
gra = automol.geom.graph(geo)
gra = automol.graph.ts_graph(gra, frm_bnd_keys=[(1, 4)], brk_bnd_keys=[(0, 1)])
gra = automol.graph.set_stereo_from_geometry(gra, geo)

In [3]:
# Build reaction object
rcts_gra = automol.graph.ts.reactants_graph(gra, stereo=False)
prds_gra = automol.graph.ts.products_graph(gra, stereo=False)
rcts_keys = sorted(map(sorted, automol.graph.connected_components_atom_keys(rcts_gra)))
prds_keys = sorted(map(sorted, automol.graph.connected_components_atom_keys(prds_gra)))
rxn = automol.reac.from_data(
    tsg=gra,
    rcts_keys=rcts_keys,
    prds_keys=prds_keys,
    cla=automol.const.ReactionClass.RING_FORM_SCISSION,
)

In [4]:
# Update reaction object structure
grxn = automol.reac.with_structures(rxn, "geom")
grxn = automol.reac.update_structures(grxn, ts_struc=geo)
zrxn = automol.reac.with_structures(grxn, "zmat")
zrxn = automol.reac.reset_conversion_info(zrxn)
grxn = automol.reac.with_structures(zrxn, "geom", ignore_zc=True)

geo = automol.reac.ts_structure(grxn)
ggra = automol.reac.ts_graph(grxn)
automol.graph.display(ggra, label=True)
automol.geom.display(geo, gra=ggra)

zma = automol.reac.ts_structure(zrxn)
zgra = automol.reac.ts_graph(zrxn)
automol.zmat.display(zma, gra=zgra)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:
geo_path = opt_path / geo_name
zma_path = opt_path / zma_name
zrxn_path = opt_path / zrxn_name
geo_path.write_text(automol.geom.xyz_string(geo))
zma_path.write_text(automol.zmat.string(zma, one_indexed=True))
zrxn_path.write_text(automol.reac.string(zrxn))

5307